In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
from pyproj import Transformer

# Function to compute hillshade from DEM
def hillshade(array, azimuth=315, angle_altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(angle_altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    return 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization

# Load the DEM raster (used for all subplots)
raster_path = "Dem_5m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem_bounds = src.bounds  # Get raster boundaries
    dem_nodata = src.nodata  # Retrieve the no-data value for the DEM

# Convert no-data values to NaN
dem = np.where(dem == dem_nodata, np.nan, dem)

# Convert DEM bounds from UTM to lat/lon using Transformer
transformer = Transformer.from_crs(src.crs, "EPSG:4326", always_xy=True)
lon_min, lat_min = transformer.transform(dem_bounds.left, dem_bounds.bottom)
lon_max, lat_max = transformer.transform(dem_bounds.right, dem_bounds.top)

# Compute hillshade from the DEM (this will be reused in all subplots)
hillshade_data = hillshade(dem)

# Load the landslide susceptibility shapefile
shapefile_path = "Results_GIS/allProbs_terrain_CF.shp"
shapefile_data = gpd.read_file(shapefile_path)

# Reproject the shapefile to lat/lon (EPSG:4326)
shapefile_data = shapefile_data.to_crs(epsg=4326)

# Define the columns to plot
columns = {
    "DS": "DensNor_DS",
    "DF": "DensNor_DF",
    "ES": "DensNor_ES",
    "EF": "DensNor_EF",
    "RS": "DensNor_RS"
}

# Custom colormap: light grey for 0, medium red for small positive values, dark red for max values
cmap = mcolors.LinearSegmentedColormap.from_list(
    "custom_grey_red_scale",
    [(0.0, "#dcdcdc"),   # Light grey for zero values
     (0.000001, "#ff6666"),  # Medium red for small positive values
     (1.0, "#4b0000")],  # Dark red for maximum value
    N=100  # Number of color segments
)

# Normalization: ensure values of 0 are grey and positive values use the color scale
norm = mcolors.Normalize(vmin=0, vmax=shapefile_data[list(columns.values())].max().max())

# Function to format degrees and minutes ticks
def format_degrees(x, pos):
    degrees = int(x)
    minutes = int((x - degrees) * 60)
    return f"{degrees}°{minutes}'"

# Function to save each susceptibility map as a TIFF file
def save_susceptibility_map(column_key, column_name):
    fig, ax = plt.subplots(figsize=(7.5, 5), dpi=600)

    # Plot the hillshade in grayscale
    ax.imshow(hillshade_data, cmap='gray', extent=[lon_min, lon_max, lat_min, lat_max],
              zorder=1)

    # Adjust the limits of the plot to the bounds of the lat/lon shapefile
    ax.set_xlim([lon_min, lon_max])
    ax.set_ylim([lat_min, lat_max])

    # Create an alpha mask based on values (0 = more transparent, positive = less transparent)
    alpha_mask = shapefile_data[column_name].apply(lambda x: 0.5 if x == 0 else 0.8)

    # Plot the susceptibility data with dynamic transparency
    shapefile_data.plot(column=column_name, cmap=cmap, norm=norm, alpha=alpha_mask, ax=ax, legend=False, zorder=2)

    # Add axis ticks in degrees without labels
    if column_key in ["DF", "EF"]:  # Y-axis ticks for DS, ES, RS
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1/3))  # 20-minute intervals
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_degrees))
        ax.tick_params(axis='y', which='both', direction='out', length=5, width=0.5, colors='black', labelleft=False)  # Remove labels
    else:
        ax.set_yticks([])  # Remove y-axis ticks for other columns
        ax.set_yticklabels([])

    if column_key in ["EF", "RS"]:  # X-axis ticks for EF, RS
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1/2))  # 30-minute intervals
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(format_degrees))
        ax.tick_params(axis='x', which='both', direction='out', length=5, width=0.5, colors='black', labelbottom=False)  # Remove labels
    else:
        ax.set_xticks([])  # Remove x-axis ticks for other columns
        ax.set_xticklabels([])

    # Set the title inside each plot
    ax.text(0.98, 0.98, column_key, transform=ax.transAxes, fontsize=16,
            verticalalignment='top', horizontalalignment='right', 
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    # Save the plot as TIFF
    fig.savefig(f"{column_key}_density_map.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Save the legend for the colorbar separately
def save_colorbar_legend():
    fig, ax = plt.subplots(figsize=(8, 1), dpi=500)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal')
    cbar.set_label('Landslide density per SU', fontsize=14, fontname='Times New Roman')
    ax.axis('off')  # Hide the axis for a clean colorbar
    fig.savefig("density_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Loop through the columns and save the maps
for key, column in columns.items():
    save_susceptibility_map(key, column)

# Save the colorbar legend
save_colorbar_legend()


In [ ]:
# good one without ticks

import os 
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker

# Ensure output directory exists
output_dir = "Plots"
#os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it doesn't already exist

# Function to compute hillshade from DEM
def compute_hillshade(array, azimuth=315, altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x * x + y * y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    hillshade_normalized = 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization
    hillshade_normalized = np.where(np.isnan(hillshade_normalized), 255, hillshade_normalized)
    return hillshade_normalized.astype(np.uint8)

# Load the DEM raster and compute hillshade
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem = np.where(dem == src.nodata, np.nan, dem)  # Handle NoData values
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Compute realistic hillshade from DEM
hillshade_data = compute_hillshade(dem)

# Load the landslide susceptibility shapefile and study area boundary shapefile
density_file = "Results_GIS/allProbs_terrain_CF.shp"
density = gpd.read_file(density_file)  # Density data with required fields

study_area_path = "Study_area.shp"
study_area = gpd.read_file(study_area_path)  # Study area shapefile

# Define the columns to plot
columns = {
    "DS": "DensNor_DS",
    "DF": "DensNor_DF",
    "ES": "DensNor_ES",
    "EF": "DensNor_EF",
    "RS": "DensNor_RS"
}

# Custom colormap: light grey for 0, medium red for small positive values, dark red for max values
cmap = mcolors.LinearSegmentedColormap.from_list(
    "custom_grey_red_scale",
    [(0.0, "#dcdcdc"),   # Light grey for zero values
     (0.000001, "#ff6666"),  # Medium red for small positive values
     (1.0, "#4b0000")],  # Dark red for maximum value
    N=100  # Number of color segments
)

# Normalization: ensure values of 0 are grey and positive values use the color scale
norm = mcolors.Normalize(vmin=0, vmax=density[list(columns.values())].max().max())

# Get the bounds of the study area to use for the extent of the plot
shape_bounds = study_area.total_bounds  # This gives [xmin, ymin, xmax, ymax]

# Function to save each susceptibility map as a TIFF file
def save_susceptibility_map(column_key, column_name):
    fig, ax = plt.subplots(figsize=(7.5, 5), dpi=600)

    # Set font properties for Times New Roman and size 18
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams["font.size"] = 16

    # Plot the realistic hillshade
    ax.imshow(hillshade_data, extent=extent, cmap='gray', zorder=1)  # 'gray' gives a standard hillshade appearance

    # Adjust the limits of the plot to the bounds of the shapefile
    ax.set_xlim([shape_bounds[0], shape_bounds[2]])
    ax.set_ylim([shape_bounds[1], shape_bounds[3]])

    # Create an alpha mask based on values (0 = more transparent, positive = less transparent)
    alpha_mask = density[column_name].apply(lambda x: 0.2 if x == 0 else 1)

    # Plot the susceptibility data with dynamic transparency
    density.plot(column=column_name, cmap=cmap, norm=norm, alpha=alpha_mask, ax=ax, legend=False, zorder=2)

    # Add the study area boundary in red
    study_area.boundary.plot(ax=ax, color='red', linewidth=0.5, zorder=3)

    # # Add axis ticks in meters (every 100,000 meters) for UTM coordinates
    # if column_key in ["DF", "EF"]:  # Y-axis ticks
    #     ax.yaxis.set_major_locator(mticker.MultipleLocator(50000))  # 100,000-meter intervals for y-axis
    #     ax.tick_params(axis='y', which='both', direction='out', length=5, width=0.5, colors='black', labelleft=False)
    # else:
    #     ax.set_yticks([])

    # if column_key in ["EF", "RS"]:  # X-axis ticks
    #     ax.xaxis.set_major_locator(mticker.MultipleLocator(50000))  # 100,000-meter intervals for x-axis
    #     ax.tick_params(axis='x', which='both', direction='out', length=5, width=0.5, colors='black', labelbottom=False)
    # else:
        # ax.set_xticks([])

    # Remove all ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    # Set the title inside each plot
    ax.text(0.5, 0.95, column_key, transform=ax.transAxes, fontsize=20,
            verticalalignment='top', horizontalalignment='center',  
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

    # Save the plot as TIFF in the "Shapefile_Images" folder
    fig.savefig(os.path.join(output_dir, f"{column_key}_density_map_32632.tif"), format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Save the legend for the colorbar separately
def save_colorbar_legend():
    fig, ax = plt.subplots(figsize=(8, 1), dpi=600)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal')
    cbar.set_label('Landslide density per SU', fontsize=18, fontname='Times New Roman')
    ax.axis('off')  # Hide the axis for a clean colorbar
    fig.savefig(os.path.join(output_dir, "density_legend_32632.tif"), format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Loop through the columns and save the maps
for key, column in columns.items():
    save_susceptibility_map(key, column)

# Save the colorbar legend
save_colorbar_legend()

In [4]:
import os 
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker

# Ensure output directory exists
output_dir = "Plots"
#os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it doesn't already exist

# Function to compute hillshade from DEM
def compute_hillshade(array, azimuth=315, altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x * x + y * y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    hillshade_normalized = 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization
    hillshade_normalized = np.where(np.isnan(hillshade_normalized), 255, hillshade_normalized)
    return hillshade_normalized.astype(np.uint8)

# Load the DEM raster and compute hillshade
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem = np.where(dem == src.nodata, np.nan, dem)  # Handle NoData values
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Compute realistic hillshade from DEM
hillshade_data = compute_hillshade(dem)

# Load the landslide susceptibility shapefile and study area boundary shapefile
density_file = "Results_GIS/allProbs_terrain_CF.shp"
density = gpd.read_file(density_file)  # Density data with required fields

study_area_path = "Study_area.shp"
study_area = gpd.read_file(study_area_path)  # Study area shapefile

# Define the columns to plot
columns = {
    "DS": "DensNor_DS",
    "DF": "DensNor_DF",
    "ES": "DensNor_ES",
    "EF": "DensNor_EF",
    "RS": "DensNor_RS"
}

# Custom colormap: light grey for 0, medium red for small positive values, dark red for max values
cmap = mcolors.LinearSegmentedColormap.from_list(
    "custom_grey_red_scale",
    [(0.0, "#dcdcdc"),   # Light grey for zero values
     (0.000001, "#ff6666"),  # Medium red for small positive values
     (1.0, "#4b0000")],  # Dark red for maximum value
    N=100  # Number of color segments
)

# Normalization: ensure values of 0 are grey and positive values use the color scale
norm = mcolors.Normalize(vmin=0, vmax=density[list(columns.values())].max().max())

# Get the bounds of the study area to use for the extent of the plot
shape_bounds = study_area.total_bounds  # This gives [xmin, ymin, xmax, ymax]
import pyproj
from shapely.geometry import box

# Define the UTM Zone 32 North projection (EPSG:32632)
proj_utm = pyproj.Proj("epsg:32632")

# Convert lat/lon bounds to UTM
lon_min_zoom, lon_max_zoom = 11.58, 11.85
lat_min_zoom, lat_max_zoom = 44.18, 44.28

x_min_zoom, y_min_zoom = proj_utm(lon_min_zoom, lat_min_zoom)
x_max_zoom, y_max_zoom = proj_utm(lon_max_zoom, lat_max_zoom)

# Define zoomed extent in UTM
zoomed_extent = [x_min_zoom, x_max_zoom, y_min_zoom, y_max_zoom]

# Update the save_susceptibility_map function to apply zoomed extent
def save_susceptibility_map_zoom(column_key, column_name):
    fig, ax = plt.subplots(figsize=(7.5, 5), dpi=600)

    # Set font properties
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams["font.size"] = 16

    # Plot the realistic hillshade
    ax.imshow(hillshade_data, extent=extent, cmap='gray', zorder=1)

    # Adjust the limits of the plot to the zoomed extent
    ax.set_xlim([zoomed_extent[0], zoomed_extent[1]])
    ax.set_ylim([zoomed_extent[2], zoomed_extent[3]])

    # Create an alpha mask based on values
    alpha_mask = density[column_name].apply(lambda x: 0.2 if x == 0 else 0.9)

    # Plot the susceptibility data with dynamic transparency
    density.plot(column=column_name, cmap=cmap, norm=norm, alpha=alpha_mask, ax=ax, legend=False, zorder=2)

    # Add the study area boundary
    study_area.boundary.plot(ax=ax, color='red', linewidth=0.5, zorder=3)

    # Remove all ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Save the plot
    fig.savefig(os.path.join(output_dir, f"{column_key}_zoomed_density_map_32632.tif"), format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Loop through the columns and save the zoomed maps
for key, column in columns.items():
    save_susceptibility_map_zoom(key, column)
